![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [493]:
import numpy as np
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Start coding here...

In [494]:
dfList = [brands,finance,info,reviews]
for df in dfList:
    df.index = df['product_id'] 

In [495]:
df = pd.concat(dfList,axis=1)
df.dropna(how='any',inplace=True)
df.head()

,product_id,brand,product_id,listing_price,sale_price,discount,revenue,product_name,product_id,description,product_id,rating,reviews
product_id,,,,,,,,,,,,,
G27341,G27341,Adidas,G27341,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",G27341,3.3,24.0
CM0081,CM0081,Adidas,CM0081,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,CM0081,2.6,37.0
B44832,B44832,Adidas,B44832,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",B44832,4.1,35.0
D98205,D98205,Adidas,D98205,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,D98205,3.5,72.0
B75586,B75586,Adidas,B75586,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,B75586,1.0,45.0


In [496]:
pricequartiles= df['listing_price'].quantile([0.25,0.5,0.75,1])
df.loc[df['listing_price'] <= pricequartiles[1],'price_label'] = 'Elite'
df.loc[df['listing_price'] <= pricequartiles[0.75],'price_label'] = 'Expensive'
df.loc[df['listing_price'] <= pricequartiles[0.5],'price_label'] = 'Average'
df.loc[df['listing_price'] <= pricequartiles[0.25],'price_label'] = 'Budget'
df.head()

,product_id,brand,product_id,listing_price,sale_price,discount,revenue,product_name,product_id,description,product_id,rating,reviews,price_label
product_id,,,,,,,,,,,,,,
G27341,G27341,Adidas,G27341,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",G27341,3.3,24.0,Expensive
CM0081,CM0081,Adidas,CM0081,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,CM0081,2.6,37.0,Budget
B44832,B44832,Adidas,B44832,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",B44832,4.1,35.0,Expensive
D98205,D98205,Adidas,D98205,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,D98205,3.5,72.0,Expensive
B75586,B75586,Adidas,B75586,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,B75586,1.0,45.0,Average


In [497]:
def convertLabel(value):
    if value <= pricequartiles[0.25]:
        return 'Budget'
    elif value >= pricequartiles[0.25] and value <= pricequartiles[0.5]:
        return 'Average'
    elif value >= pricequartiles[0.5] and value <=pricequartiles[0.75]:
        return 'Expensive'
    else:
        return 'Elite'

In [498]:
group = df.groupby(by=['brand','price_label'])
mean_revenue = group['revenue'].mean().round(2)
num_products = group['price_label'].count().round(2)
mean_price = group['listing_price'].mean().round(2)
price_label = mean_price.apply(convertLabel)
mean_revenue

adidas_vs_nike = pd.DataFrame({
    'price_label':price_label,
    'num_products':num_products,
    'mean_revenue':mean_revenue
})
brand = []
for tup in list(adidas_vs_nike.index):
    brand.append(tup[0])
brand = pd.Series(brand)
brand.index = adidas_vs_nike.index
adidas_vs_nike['brand'] = brand
adidas_vs_nike = adidas_vs_nike[['brand','price_label','num_products','mean_revenue']] 


In [499]:
def splitDesc(text):
    binLen = 100
    bins = []
    for i in range(0,len(text),binLen):
        bins.append(text[i:i+binLen])
    return bins
def lenBin(bins):
    return len(bins)
splited_desc = df['description'].apply(splitDesc)
df['desc_len'] = splited_desc.apply(lenBin)

In [500]:
lab = [i for i in range(2,8)]
df.loc[df['desc_len'] <= 100, 'desc_len_label'] = str(1*100)
for i, num in enumerate(lab):
    df.loc[(df['desc_len'] >= num), 'desc_len_label'] = str(num*100)
df

,product_id,brand,product_id,listing_price,sale_price,discount,revenue,product_name,product_id,description,product_id,rating,reviews,price_label,desc_len,desc_len_label
product_id,,,,,,,,,,,,,,,,
G27341,G27341,Adidas,G27341,75.99,37.99,0.5,1641.17,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",G27341,3.3,24.0,Expensive,2,200
CM0081,CM0081,Adidas,CM0081,9.99,5.99,0.4,398.93,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,CM0081,2.6,37.0,Budget,2,200
B44832,B44832,Adidas,B44832,69.99,34.99,0.5,2204.37,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",B44832,4.1,35.0,Expensive,3,300
D98205,D98205,Adidas,D98205,79.99,39.99,0.5,5182.70,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,D98205,3.5,72.0,Expensive,3,300
B75586,B75586,Adidas,B75586,47.99,19.20,0.6,1555.20,Women's adidas Sport Inspired Duramo Lite 2.0 ...,B75586,Refine your interval training in these women's...,B75586,1.0,45.0,Average,3,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AT6100-606,AT6100-606,Nike,AT6100-606,0.00,64.95,0.0,0.00,Nike Tiempo Legend 8 Academy TF,AT6100-606,The Nike Tiempo Legend 8 Academy TF takes the ...,AT6100-606,0.0,0.0,Budget,2,200
CT9155-063,CT9155-063,Nike,CT9155-063,0.00,139.95,0.0,251.91,Nike React Metcon AMP,CT9155-063,The Nike React Metcon AMP takes the stability ...,CT9155-063,3.0,1.0,Budget,4,400
CI1236-100,CI1236-100,Nike,CI1236-100,159.95,127.97,0.0,230.35,Air Jordan 8 Retro,CI1236-100,The Air Jordan 8 Retro recaptures the memorabl...,CI1236-100,5.0,1.0,Elite,3,300


In [501]:
group = df.groupby(by=['desc_len_label'])
mean_rating = group['rating'].mean().round(2)
num_reviews = group['reviews'].count()
description_length = group['desc_len'].sum()

In [502]:
description_lengths = pd.DataFrame({
    'description_length':description_length.index,
    'mean_rating':mean_rating,
    'num_reviews':num_reviews,
})
description_lengths

,description_length,mean_rating,num_reviews
desc_len_label,,,
100,100,2.26,7
200,200,3.19,526
300,300,3.28,1785
400,400,3.29,651
500,500,3.35,118
600,600,3.12,15
700,700,3.65,15
